# 🍔 Food Delivery Data Integration

This notebook integrates data from three different sources:
- **orders.csv** - Transactional order data
- **users.json** - User master data
- **restaurants.sql** - Restaurant master data

We'll merge these datasets and create a unified DataFrame for analysis.

## Step 1: Import Libraries

In [2]:
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Step 2: Load CSV Data (Orders)

In [3]:
# Load orders data from CSV
orders_df = pd.read_csv('orders.csv')

print(f"📊 Orders DataFrame loaded!")
print(f"   Shape: {orders_df.shape}")
print(f"   Columns: {list(orders_df.columns)}")
print("\n🔍 First 5 rows:")
orders_df.head()

📊 Orders DataFrame loaded!
   Shape: (10000, 6)
   Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']

🔍 First 5 rows:


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


## Step 3: Load JSON Data (Users)

In [4]:
# Load users data from JSON
with open('users.json', 'r') as f:
    users_data = json.load(f)

# Convert to DataFrame
users_df = pd.DataFrame(users_data)

print(f"📊 Users DataFrame loaded!")
print(f"   Shape: {users_df.shape}")
print(f"   Columns: {list(users_df.columns)}")
print("\n🔍 First 5 rows:")
users_df.head()

📊 Users DataFrame loaded!
   Shape: (3000, 4)
   Columns: ['user_id', 'name', 'city', 'membership']

🔍 First 5 rows:


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


## Step 4: Load SQL Data (Restaurants)

In [6]:
# Parse SQL INSERT statements to extract restaurant data
def parse_sql_file(filename):
    """Parse SQL INSERT statements and return a DataFrame"""
    with open(filename, 'r') as f:
        sql_content = f.read()
    
    # Extract all INSERT statements
    pattern = r"INSERT INTO restaurants VALUES \((\d+), '([^']+)', '([^']+)', ([\d.]+)\);"
    matches = re.findall(pattern, sql_content)
    
    # Create DataFrame
    data = []
    for match in matches:
        data.append({
            'restaurant_id': int(match[0]),
            'restaurant_name': match[1],
            'cuisine': match[2],
            'rating': float(match[3])
        })
    
    return pd.DataFrame(data)

# Load restaurants data
restaurants_df = parse_sql_file('restaurants.sql')

print(f"📊 Restaurants DataFrame loaded!")
print(f"   Shape: {restaurants_df.shape}")
print(f"   Columns: {list(restaurants_df.columns)}")
print("\n🔍 First 5 rows:")
restaurants_df.head()

📊 Restaurants DataFrame loaded!
   Shape: (500, 4)
   Columns: ['restaurant_id', 'restaurant_name', 'cuisine', 'rating']

🔍 First 5 rows:


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


## Step 5: Data Quality Check

In [9]:
print("📋 Data Quality Summary:\n")

print("Orders DataFrame:")
print(orders_df.info())
print(f"\nMissing values:\n{orders_df.isnull().sum()}\n")
print("="*50)

print("\nUsers DataFrame:")
print(users_df.info())
print(f"\nMissing values:\n{users_df.isnull().sum()}\n")
print("="*50)

print("\nRestaurants DataFrame:")
print(restaurants_df.info())
print(f"\nMissing values:\n{restaurants_df.isnull().sum()}")

📋 Data Quality Summary:

Orders DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB
None

Missing values:
order_id           0
user_id            0
restaurant_id      0
order_date         0
total_amount       0
restaurant_name    0
dtype: int64


Users DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name   

## Step 6: Merge the Datasets

We'll perform **left joins** to retain all orders:
1. Merge orders with users on `user_id`
2. Merge result with restaurants on `restaurant_id`

In [16]:
# Step 1: Merge orders with users
print("🔗 Merging orders with users...")
merged_df = orders_df.merge(users_df, on='user_id', how='left')
print(f"   After user merge: {merged_df.shape}")

# Step 2: Merge with restaurants (using suffixes to distinguish duplicate columns)
print("🔗 Merging with restaurants...")
final_dataset = merged_df.merge(
    restaurants_df, 
    on='restaurant_id', 
    how='left',
    suffixes=('_order', '_restaurant')
)
print(f"   After restaurant merge: {final_dataset.shape}")

print("\n✅ Final dataset created successfully!")
print(f"\n📊 Final Dataset Shape: {final_dataset.shape}")
print(f"   Total Orders: {len(final_dataset)}")
print(f"   Total Columns: {len(final_dataset.columns)}")
print(f"\nColumns: {list(final_dataset.columns)}")
print("\n💡 Note: 'restaurant_name_order' is from orders.csv, 'restaurant_name_restaurant' is from restaurants.sql")

🔗 Merging orders with users...
   After user merge: (10000, 9)
🔗 Merging with restaurants...
   After restaurant merge: (10000, 12)

✅ Final dataset created successfully!

📊 Final Dataset Shape: (10000, 12)
   Total Orders: 10000
   Total Columns: 12

Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_order', 'name', 'city', 'membership', 'restaurant_name_restaurant', 'cuisine', 'rating']

💡 Note: 'restaurant_name_order' is from orders.csv, 'restaurant_name_restaurant' is from restaurants.sql


## Step 7: Preview Final Dataset

In [20]:
# Display first 10 rows
final_dataset.head(5)

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_order,name,city,membership,restaurant_name_restaurant,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [22]:
# Display dataset info
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   order_id                    10000 non-null  int64  
 1   user_id                     10000 non-null  int64  
 2   restaurant_id               10000 non-null  int64  
 3   order_date                  10000 non-null  object 
 4   total_amount                10000 non-null  float64
 5   restaurant_name_order       10000 non-null  object 
 6   name                        10000 non-null  object 
 7   city                        10000 non-null  object 
 8   membership                  10000 non-null  object 
 9   restaurant_name_restaurant  10000 non-null  object 
 10  cuisine                     10000 non-null  object 
 11  rating                      10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [25]:
# Statistical summary
final_dataset.describe()

,order_id,user_id,restaurant_id,total_amount,rating
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1504.117700,251.016700,801.162412,4.045430
std,2886.89568,861.727776,144.622558,405.458753,0.606531
min,1.00000,1.000000,1.000000,100.200000,3.000000
25%,2500.75000,761.000000,127.000000,446.310000,3.500000
50%,5000.50000,1508.000000,251.000000,806.295000,4.100000
75%,7500.25000,2250.250000,376.000000,1149.227500,4.600000
max,10000.00000,3000.000000,500.000000,1499.830000,5.000000


## Step 8: Save Final Dataset (Optional)

In [27]:
# Uncomment to save the final dataset to CSV
final_dataset.to_csv('final_food_delivery_dataset.csv', index=False)
# print("✅ Final dataset saved to 'final_food_delivery_dataset.csv'")

---

## 🎯 Your Analysis Here

The `final_dataset` DataFrame is ready for your exploratory data analysis!

**Happy Analyzing! 🚀**

Which city has the highest total revenue (total_amount) from Gold members?  
Hyderabad
Bangalore
Chennai
Pune

In [28]:
# Your custom analysis here
import pandas as pandas_object
import numpy as numpy_object

In [29]:
final_food_delivery_dataset = pandas_object.read_csv('final_food_delivery_dataset.csv')
final_food_delivery_dataset.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_order,name,city,membership,restaurant_name_restaurant,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


Which city has the highest total revenue (total_amount) from Gold members?  

In [32]:
# Filter only Gold members
gold_df = final_food_delivery_dataset[final_food_delivery_dataset["membership"] == "Gold"]

# Group by city and sum revenue
revenue_by_city = gold_df.groupby("city")["total_amount"].sum()

# Sort descending
revenue_by_city = revenue_by_city.sort_values(ascending=False)

print(revenue_by_city)

# Highest revenue city
top_city = revenue_by_city.idxmax()
print("\nCity with highest Gold member revenue:", top_city)

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

City with highest Gold member revenue: Chennai


Which cuisine has the highest average order value across all orders?

In [33]:
aov_by_cuisine = (
    final_food_delivery_dataset.groupby("cuisine")["total_amount"]
      .mean()
      .sort_values(ascending=False)
)

print(aov_by_cuisine)

print("\nHighest AOV cuisine:", aov_by_cuisine.idxmax())

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

Highest AOV cuisine: Mexican


How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [34]:
# Total spend per user
user_totals = final_food_delivery_dataset.groupby("user_id")["total_amount"].sum()

# Count users spending > 1000
count_users = (user_totals > 1000).sum()

print("Users with total spend > ₹1000:", count_users)

Users with total spend > ₹1000: 2544


Which restaurant rating range generated the highest total revenue?

In [35]:
# Create rating bins
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_food_delivery_dataset["rating_range"] = pandas_object.cut(final_food_delivery_dataset["rating"], bins=bins, labels=labels, include_lowest=True)

# Revenue by rating range
revenue = final_food_delivery_dataset.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)

print(revenue)

print("\nHighest revenue range:", revenue.idxmax())

rating_range
4.6–5.0    2197030.75
3.0–3.5    2136772.70
4.1–4.5    1960326.26
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

Highest revenue range: 4.6–5.0


C:\Users\mrudu\AppData\Local\Temp\ipykernel_26360\3618869438.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  revenue = final_food_delivery_dataset.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


In [36]:
Among Gold members, which city has the highest average order value?

Object `value` not found.


In [37]:
gold = final_food_delivery_dataset[final_food_delivery_dataset["membership"] == "Gold"]

aov_city = (
    gold.groupby("city")["total_amount"]
        .mean()
        .sort_values(ascending=False)
)

print(aov_city)
print("\nHighest AOV city:", aov_city.idxmax())

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

Highest AOV city: Chennai


Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [38]:
summary = (
    final_food_delivery_dataset.groupby("cuisine")
      .agg(
          distinct_restaurants=("restaurant_id", "nunique"),
          total_revenue=("total_amount", "sum")
      )
      .sort_values("distinct_restaurants")   # lowest first
)

print(summary)

         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65
Indian                    126     1971412.58
Italian                   126     2024203.80
Mexican                   128     2085503.09


What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [40]:
total_orders = len(final_food_delivery_dataset)
gold_orders = final_food_delivery_dataset[final_food_delivery_dataset["membership"] == "Gold"].shape[0]

percentage = round((gold_orders / total_orders) * 100)

print("Gold Orders:", gold_orders)
print("Total Orders:", total_orders)
print("Gold %:", percentage)


Gold Orders: 4987
Total Orders: 10000
Gold %: 50


Which restaurant has the highest average order value but less than 20 total orders?

In [42]:
print(
    final_food_delivery_dataset.groupby("restaurant_name_restaurant")["total_amount"]
      .agg(['count','mean'])
      .query('count < 20')
      .sort_values('mean', ascending=False)
)

                            count         mean
restaurant_name_restaurant                    
Restaurant_294                 13  1040.222308
Restaurant_262                 18  1029.473333
Restaurant_77                  12  1029.180833
Restaurant_193                 15  1026.306667
Restaurant_7                   16  1002.140625
...                           ...          ...
Restaurant_184                 19   621.828947
Restaurant_498                 18   596.815556
Restaurant_192                 14   589.972857
Restaurant_329                 15   578.578667
Restaurant_300                 17   572.686471

[241 rows x 2 columns]


Which combination contributes the highest revenue?

In [44]:
revenue_combo = (
    final_food_delivery_dataset.groupby(["membership", "cuisine"])["total_amount"]
      .sum()
      .sort_values(ascending=False)
)

print(revenue_combo)
print("\nHighest revenue combination:", revenue_combo.idxmax())

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

Highest revenue combination: ('Regular', 'Mexican')


During which quarter of the year is the total revenue highest?

In [45]:
final_food_delivery_dataset["order_date"] = pandas_object.to_datetime(final_food_delivery_dataset["order_date"])

revenue_by_quarter = (
    final_food_delivery_dataset.groupby(final_food_delivery_dataset["order_date"].dt.quarter)["total_amount"]
      .sum()
      .sort_values(ascending=False)
)

print(revenue_by_quarter)

print("\nHighest revenue quarter: Q" + str(revenue_by_quarter.idxmax()))

order_date
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

Highest revenue quarter: Q3


C:\Users\mrudu\AppData\Local\Temp\ipykernel_26360\2523300662.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_food_delivery_dataset["order_date"] = pandas_object.to_datetime(final_food_delivery_dataset["order_date"])
